### 0. Imports

In [1]:
%load_ext autoreload
%autoreload 2

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
import polars as pl

# Visualizaciones
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

# Vigilar progreso bucles
# -----------------------------------------------------------------------
from tqdm import tqdm

# Gestionar los warnings
# -----------------------------------------------------------------------
import warnings

# modificar el path
# -----------------------------------------------------------------------
import sys
sys.path.append("..")

# importar funciones de soporte
# -----------------------------------------------------------------------
import src.soporte_eda as se
import src.soporte_preprocesamiento as sp

# evaluar objetos literales
# -----------------------------------------------------------------------
from ast import literal_eval 

# statistics functions
# -----------------------------------------------------------------------
from scipy.stats import pearsonr, spearmanr, pointbiserialr


# 1. Introduction - Madrid rental prices - Cleaning

# 2. Preliminary analysis and data cleaning

## 2.1 Import data

In [3]:
pd.set_option("display.max_columns",40)
rent_df = pd.read_csv("../datos/api_rent_madrid.csv",sep=";")
rent_df.head()

,propertyCode,numPhotos,price,propertyType,operation,size,exterior,rooms,bathrooms,address,province,municipality,country,latitude,longitude,showAddress,distance,description,hasVideo,status,newDevelopment,priceByArea,detailedType,suggestedTexts,hasPlan,has3DTour,has360,hasStaging,superTopHighlight,topNewDevelopment,externalReference,floor,district,neighborhood,hasLift,parkingSpace,labels,newDevelopmentFinished
0,98903350,33,550.0,chalet,rent,371.0,False,6,3,Numancia de la Sagra,Toledo,Numancia de la Sagra,es,400.675.955,-38.225.891,False,40116,UGENA VILLAGE INMOBILIARIA GESTIONA en exclusi...,False,good,False,1.0,"{'typology': 'chalet', 'subTypology': 'indepen...","{'subtitle': 'Numancia de la Sagra', 'title': ...",False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,91615378,12,750.0,flat,rent,60.0,True,1,1,"Calle de Alejandro Villegas, 23",Madrid,Madrid,es,404.646.164,-36.489.179,True,7037,LOFT DE DISEÑO. Orientación Este. Muy soleado ...,False,good,False,13.0,{'typology': 'flat'},"{'subtitle': 'Canillas, Madrid', 'title': 'Pis...",False,False,False,False,False,False,AV23,3,Hortaleza,Canillas,True,NaN,NaN,NaN
2,88363329,21,750.0,flat,rent,70.0,True,2,1,Gregorio izquierdo,Madrid,San Sebastián de los Reyes,es,405.500.865,-36.278.085,False,16145,"Estupendo piso muy iluminado, recientemente re...",True,good,False,11.0,{'typology': 'flat'},"{'subtitle': 'Centro Urbano, San Sebastián de ...",True,False,False,False,False,False,NaN,bj,Centro Urbano,NaN,False,NaN,NaN,NaN
3,98621753,28,400.0,penthouse,rent,67.0,True,2,2,Calle Madrid,Madrid,Villamanrique de Tajo,es,40.069.197,-3.241.383,False,55041,"A estrenar ático con piscina en Calle Madrid, ...",False,good,False,6.0,"{'typology': 'flat', 'subTypology': 'penthouse'}","{'subtitle': 'Villamanrique de Tajo', 'title':...",False,False,True,False,False,False,90004,2,NaN,NaN,False,NaN,NaN,NaN
4,98222714,22,450.0,flat,rent,89.0,False,2,1,Calle Arroyo,Toledo,Recas,es,400.536.974,-39.912.159,False,47186,REDPISO alquila esta vivienda situada en el mu...,False,good,False,5.0,{'typology': 'flat'},"{'subtitle': 'Recas', 'title': 'Piso en Calle ...",False,False,False,False,False,False,RP1882022101719,2,NaN,NaN,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",NaN,NaN


## 2.2 Explore dataframe

In [4]:
pd.set_option("display.max_rows",40)
se.exploracion_dataframe(rent_df)

El número de datos es 450 y el de columnas es 38

 ..................... 

Las primeras filas del dataframe son:


,propertyCode,numPhotos,price,propertyType,operation,size,exterior,rooms,bathrooms,address,province,municipality,country,latitude,longitude,showAddress,distance,description,hasVideo,status,newDevelopment,priceByArea,detailedType,suggestedTexts,hasPlan,has3DTour,has360,hasStaging,superTopHighlight,topNewDevelopment,externalReference,floor,district,neighborhood,hasLift,parkingSpace,labels,newDevelopmentFinished
0,98903350,33,550.0,chalet,rent,371.0,False,6,3,Numancia de la Sagra,Toledo,Numancia de la Sagra,es,400.675.955,-38.225.891,False,40116,UGENA VILLAGE INMOBILIARIA GESTIONA en exclusi...,False,good,False,1.0,"{'typology': 'chalet', 'subTypology': 'indepen...","{'subtitle': 'Numancia de la Sagra', 'title': ...",False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,91615378,12,750.0,flat,rent,60.0,True,1,1,"Calle de Alejandro Villegas, 23",Madrid,Madrid,es,404.646.164,-36.489.179,True,7037,LOFT DE DISEÑO. Orientación Este. Muy soleado ...,False,good,False,13.0,{'typology': 'flat'},"{'subtitle': 'Canillas, Madrid', 'title': 'Pis...",False,False,False,False,False,False,AV23,3,Hortaleza,Canillas,True,NaN,NaN,NaN



 ..................... 

Los tipos de las columnas son:


,tipo_dato
propertyCode,int64
numPhotos,int64
price,float64
propertyType,object
operation,object
size,float64
exterior,bool
rooms,int64
bathrooms,int64
address,object



 ..................... 

Los duplicados que tenemos en el conjunto de datos son: 8

 ..................... 

Los nulos que tenemos en el conjunto de datos son:


...

,count,pct
parkingSpace,,
"{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': True}",136,30.2
"{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 50.0}",1,0.2
"{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 40.0}",1,0.2
"{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 90.0}",1,0.2
"{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 15000.0}",1,0.2


La columna LABELS tiene 1 valores únicos, de los cuales los primeros son:


,count,pct
labels,,
"[{'name': 'apartamentoType', 'text': 'Apartamento'}]",30,6.7


La columna NEWDEVELOPMENTFINISHED tiene 2 valores únicos, de los cuales los primeros son:


,count,pct
newDevelopmentFinished,,
True,12,2.7
False,5,1.1


From a quick first data types and column value inspection, we observe:

**Low variability columns**
- operation: A single value column. Should be dropped as it adds no variance.
- Country: A single value column. Should be dropped as it adds no variance.
- Status: The most frequent category has a 91.8% of observations. The second just 3.8% meaning 17 observations, which would leave very little room for an algorithm to have a significant sample for that second category. It is better dropped.
- newDevelopment: Same as Status.
- detailedtype: Most flats (76.4%) do not posses a subTypology. Thus, it is the same as having 76.4% NaN values.
- has3DTour: With just 6 samples for the binary minority class, it brings no information.
- hasStaging: A single value column. Should be dropped as it adds no variance.
- superTopHighlight: A single value column. Should be dropped as it adds no variance.
- topNewDevelopment: A single value column. Should be dropped as it adds no variance.


In [5]:
rent_df.drop(columns=["operation","country","status", "newDevelopment",
                      "detailedType","has3DTour","hasStaging","superTopHighlight",
                      "topNewDevelopment"],inplace=True)




**Duplicates**
- There are 8 row level duplicates in the dataset.


In [6]:
rent_df.drop_duplicates(inplace=True)

- There are other suposed unique values, such as the description or latitude, longitude that are duplicaded, which is worth inspecting:

In [7]:
rent_df[rent_df.duplicated(subset="description",keep=False)].head(4)

,propertyCode,numPhotos,price,propertyType,size,exterior,rooms,bathrooms,address,province,municipality,latitude,longitude,showAddress,distance,description,hasVideo,priceByArea,suggestedTexts,hasPlan,has360,externalReference,floor,district,neighborhood,hasLift,parkingSpace,labels,newDevelopmentFinished
53,98708163,13,650.0,flat,70.0,True,2,1,jerez,Toledo,Calypo Fado,40.247.822,-4.060.189,False,35607,"piso azul inmobiliaria alquila en calypo fado,...",False,9.0,"{'subtitle': 'Calypo Fado', 'title': 'Piso en ...",False,False,NaN,1,NaN,NaN,False,NaN,NaN,NaN
55,98715256,13,650.0,flat,70.0,True,2,1,jerez,Toledo,Calypo Fado,402.449.057,-40.682.374,False,36358,"piso azul inmobiliaria alquila en calypo fado,...",False,9.0,"{'subtitle': 'Calypo Fado', 'title': 'Piso en ...",False,False,NaN,1,NaN,NaN,False,NaN,NaN,NaN
78,98932280,20,650.0,flat,55.0,True,1,1,barrio Rejas,Madrid,Madrid,404.415.556,-35.836.227,False,10495,LEER TEXTO COMPLETO Apartamento totalmente amu...,False,12.0,"{'subtitle': 'Rejas, Madrid', 'title': 'Piso'}",False,False,11DEY750/11,4,San Blas,Rejas,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu...","[{'name': 'apartamentoType', 'text': 'Apartame...",NaN
85,98931699,20,650.0,flat,55.0,True,1,1,barrio Rejas,Madrid,Madrid,404.414.556,-35.860.227,False,10296,LEER TEXTO COMPLETO Apartamento totalmente amu...,False,12.0,"{'subtitle': 'Rejas, Madrid', 'title': 'Piso'}",False,False,10DEY750/10,4,San Blas,Rejas,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu...","[{'name': 'apartamentoType', 'text': 'Apartame...",NaN


It seems that except for the property code, records sharing the same description are in fact duplicated and should thus be dropped.

In [8]:
rent_df.shape[0]

442

In [9]:
rent_df.drop_duplicates(subset="description",keep="first", inplace=True)

In [10]:
rent_df.shape[0]

428

**Missing values**

Most columns have a 4% of missing values that should be treated. The following have more than 5% but less than 50% and could be interesting to keep in abscence of other errors:
- floor
- district
- hasLift

In this case, as they are of categorical nature, they can be handled straight away by creating a new 'unknown' category in the case of floor and district. In the case of hasLift, it can be inferred that if this is not specified in the listing it is due to a lack of this feature, so it can be set to false.

In [11]:
rent_df["district"].fillna("unknown",inplace=True)
rent_df["hasLift"].fillna(False,inplace=True)

C:\Users\Miguel\AppData\Local\Temp\ipykernel_20764\1949038188.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  rent_df["district"].fillna("unknown",inplace=True)
C:\Users\Miguel\AppData\Local\Temp\ipykernel_20764\1949038188.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For

The following columns should be dropped due to high volume (more than 50%) of missing values:
- neighborhood
- labels
- newDevelopmentFinished

In [12]:
rent_df.drop(columns=["neighborhood","labels","newDevelopmentFinished"],inplace=True)

The column 'parkingSpace' has above 68% of missing values, but this one being almost binary could be kept. Missing values could be kept as 'unknown', replacing a False value, as most non-missing values are True, in case it brings useful information. This is to be contrasted in subsequent steps.

In [13]:
# use pd.categorical as .astype("category") raises errors
rent_df["parkingSpace_included_in_listing"] = np.where(extract_value(rent_df['parkingSpace'], 'isParkingSpaceIncludedInPrice') == True,True,False)
rent_df.drop(columns="parkingSpace",inplace=True)

### Boolean columns values

Explore values from boolean columns:

In [14]:
for column in rent_df.select_dtypes(bool):
    
    display(rent_df[column].value_counts())

exterior
True     347
False     81
Name: count, dtype: int64

showAddress
False    333
True      95
Name: count, dtype: int64

hasLift
True     216
False    212
Name: count, dtype: int64

parkingSpace_included_in_listing
False    307
True     121
Name: count, dtype: int64

A priori, everything is in order.

### Categorical columns values
**High cardinality**
- Description: For obvious reasons. Some characteristics could be extracted, such as wether the rental unit includes furniture or not. Do not drop for now.
- SuggestedTexts: High cardinality, possible does not add any value. Drop.
- externalReference: High cardinality, possible does not add any value as it looks like an id column. Drop.
- address: it quite possibly adds no value. Leave for now in case it can serve the purpose of getting proxy variables if the model needs it.


In [15]:
rent_df.drop(columns=["suggestedTexts","externalReference"], inplace=True)

**Data types and errors**

- Latitude & Longitude: Object data types are incorrect due to typo in the coordinates. Should be float.


In [16]:
rent_df["latitude"] = rent_df["latitude"].str.replace(".","").str[:7].astype(int) / 10**5
rent_df["longitude"] = rent_df["longitude"].str.replace(".","").str[:7].astype(int) / 10**5

- HasVideo: should be boolean.
- hasPlan to has360: Should be boolean.
- hasLift: should be boolean

In [17]:
rent_df[["hasVideo","hasPlan","has360","hasLift"]] = rent_df[["hasVideo","hasPlan","has360","hasLift"]].astype(bool)

- floor: although categorical, should be in numeric format. Assuming "bj" = "bajo", "en" = "entresuelo", "st" = "sotano" and "ss" = "semisotano", then:

In [18]:
floor_replacement_dict = {
    "bj": 0,     # bajo 
    "en": 0.5,   # entresuelo
    "st": -1,    # sótano 
    "ss": -0.5   # semisótano 
}

rent_df["floor"] = rent_df["floor"].replace(floor_replacement_dict).astype("object")

###

Additional concerns like category underrepresentation will be addressed after further EDA and data understanding.

### Numerical columns

For numerical columns, the quickest way to assess value ranges and other statistics is through a descriptive summary:

In [19]:
rent_df.describe().T

,count,mean,std,min,25%,50%,75%,max
propertyCode,428.0,9.447580e+07,1.418609e+07,666696.00000,9.752670e+07,9.866600e+07,9.889510e+07,9.893968e+07
numPhotos,428.0,2.188551e+01,9.735542e+00,0.00000,1.500000e+01,2.100000e+01,2.800000e+01,7.200000e+01
price,428.0,6.726963e+02,8.499649e+01,340.00000,6.400000e+02,7.000000e+02,7.400000e+02,7.500000e+02
size,428.0,6.779673e+01,3.636241e+01,20.00000,4.600000e+01,6.150000e+01,8.000000e+01,3.710000e+02
rooms,428.0,1.693925e+00,9.857346e-01,0.00000,1.000000e+00,2.000000e+00,2.000000e+00,6.000000e+00
bathrooms,428.0,1.193925e+00,4.298690e-01,1.00000,1.000000e+00,1.000000e+00,1.000000e+00,3.000000e+00
latitude,428.0,4.030558e+01,1.766794e+00,4.03751,4.030286e+01,4.040550e+01,4.048334e+01,4.090739e+01
longitude,428.0,-3.682008e+00,3.205519e-01,-4.33449,-3.776485e+00,-3.696145e+00,-3.604645e+00,-3.577900e-01
distance,428.0,2.229495e+04,1.720615e+04,183.00000,6.991000e+03,1.963150e+04,3.516600e+04,5.991900e+04
priceByArea,412.0,1.201699e+01,5.421611e+00,1.00000,8.000000e+00,1.100000e+01,1.500000e+01,3.300000e+01


No presence of outliers is detected judging from the ranges of features like price, size and distance.

PropertyCode should be dropped, as it is just an ID column with no predictive power and no potential use.

In [20]:
rent_df.drop(columns="propertyCode", inplace=True)

Price shows a healthy range of values with no odd numbers (negative, 999 format, etc) within range that could encode missing values.

Rooms and bathroom probably should not be treated as numerical as they probably have very few unique values:


In [21]:
print(f"Rooms has {rent_df.rooms.nunique()} values. Bathrooms has {rent_df.bathrooms.nunique()}.")

# convert to object to be treated as category
rent_df["rooms"] = rent_df["rooms"].astype("object")
rent_df["bathrooms"] = rent_df["bathrooms"].astype("object")

Rooms has 6 values. Bathrooms has 3.



Appart from that, there is the minimum price by area of 1 euro, that looks suspiciously low. Let's check it:

In [22]:
rent_df[["price","size","address","municipality","province","priceByArea"]].sort_values("priceByArea").head(3)

,price,size,address,municipality,province,priceByArea
0,550.0,371.0,Numancia de la Sagra,Numancia de la Sagra,Toledo,1.0
415,750.0,341.0,sexta bandera,Las Ventas de Retamosa,Toledo,2.0
296,600.0,180.0,Meco,Meco,Madrid,3.0


It actually looks like 'priceByArea' has correct values, except from the fact that they should be float values. Recalculating it:

In [23]:
rent_df["priceByArea"] = rent_df["price"] / rent_df["size"]

All other values present healthy ranges. The only feature that raises an eyebrow is distance, as it is not quite clear with respect to what that distance is measured. So let's check to what other feature's values its minimum values correspond:

In [24]:
rent_df[["address","municipality","distance"]].sort_values("distance").head(2)

,address,municipality,distance
133,"Calle Carretas, 31",Madrid,183
154,"Calle de Atocha, 51",Madrid,470


The shortest distance corresponds to Calle Carretas, 31. A quick google look up returns that it is close to "Kilometer 0" in Puerta del Sol, and the distance coincides with 183m. 

This distance measure is rather brute, and probably not as informative as the median price of living in certain districts or streets as it informs merely about distance to the center of Madrid. However, let's keep it for now to analyse it via due EDA.

# 2. Export for EDA

Data has been preliminarily explored and cleaned. The resulting dataset will be exported to continue with EDA in `notebooks/1_exploration_v1.ipynb`

In [25]:
rent_df.columns

Index(['numPhotos', 'price', 'propertyType', 'size', 'exterior', 'rooms',
       'bathrooms', 'address', 'province', 'municipality', 'latitude',
       'longitude', 'showAddress', 'distance', 'description', 'hasVideo',
       'priceByArea', 'hasPlan', 'has360', 'floor', 'district', 'hasLift',
       'parkingSpace_included_in_listing'],
      dtype='object')

In [32]:
rent_df_cleaned = rent_df[['numPhotos', 'price', 'propertyType', 'size', 'exterior', 'rooms',
       'bathrooms', 'distance', 'priceByArea', 'hasPlan', 'has360', 'hasVideo', 'description',
       'hasLift', 'parkingSpace_included_in_listing','municipality','province','floor']]

# version for first iteration of preprocessing
rent_df_cleaned.to_csv("../datos/cleaned/rent_df_cleaned_v1.csv")